In [1]:
import pandas as pd
import sklearn
import numpy as np

## Data Prep

Import the screening results from Reviewer1, Reviewer2 and GPT.

Each reviewer evaluated 100 abstracts and answered `YES` or `NO` to two parameters: **AI_ML** and **NLP**. We extract the answers for each parameter and reviewer as python lists and convert `YES`/`NO` to `1` and `0` for easier manipulation.

In [2]:
def prepare_for_kappa(lst):
    lst = [value for value in lst if value != "MISSING"]
    return [1 if value == "YES" else 0 for value in lst]

In [3]:
df_reviewer1 = pd.read_excel('Screening_reviewer_1.xlsx')
lst_reviewer1_aiml = prepare_for_kappa(df_reviewer1.iloc[:102]['AI_ML'].tolist())
lst_reviewer1_nlp = prepare_for_kappa(df_reviewer1.iloc[:102]['NLP'].tolist())
# assert that there are 100 answers
len(lst_reviewer1_aiml)

100

In [4]:
df_reviewer2 = pd.read_excel('Screening_reviewer_2.xlsx')
lst_reviewer2_aiml = prepare_for_kappa(df_reviewer2.iloc[:102]['AI_ML'].tolist())
lst_reviewer2_nlp = prepare_for_kappa(df_reviewer2.iloc[:102]['NLP'].tolist())
# assert that there are 100 answers
len(lst_reviewer2_aiml)

100

In [5]:
df_reviewer_gpt = pd.read_excel('Screening_reviewer_GPT4.xlsx')
lst_reviewer_gpt_aiml = prepare_for_kappa(df_reviewer_gpt.iloc[:102]['AI_ML'].tolist())
lst_reviewer_gpt_nlp = prepare_for_kappa(df_reviewer_gpt.iloc[:102]['NLP'].tolist())
# assert that there are 100 answers
len(lst_reviewer_gpt_aiml)

100

In [6]:
df_resolution = pd.read_excel('Screening_resolution.xlsx')
lst_resolution_aiml = prepare_for_kappa(df_resolution.iloc[:102]['AI_ML_Resolution'].tolist())
lst_resolution_nlp = prepare_for_kappa(df_resolution.iloc[:102]['NLP_Resolution'].tolist())
# assert that there are 100 answers
len(lst_resolution_aiml)

100

## Calculating Inter-rater Agreement

We calculate inter-rater agreement as Cohen's Kappa coefficient. 

In [7]:
def calculate_kappa_and_CI(list1, list2):
  ''' Function that calculates agreement between two graders whose answers are list1 and list2 respectively. Source code:
      https://rowannicholls.github.io/python/statistics/agreement/cohens_kappa.html#confidence-intervals

      Parameters:
      list1, list2: lists of 1s and 0s, where 1 stands for a "YES" answer and 0 for a "NO" answer

      Returns:
      a tuple of three numerical values representing the Cohen's Kappa value, lower end of the confidence interval and
      upper end of the confidence interval, in this order.
  '''

  # Create confusion matrix
  cm = sklearn.metrics.confusion_matrix(list1, list2)

  # Sample size
  n = np.sum(cm)

  # Expected matrix
  sum0 = np.sum(cm, axis=0)
  sum1 = np.sum(cm, axis=1)
  expected = np.outer(sum0, sum1) / n

  # Number of classes
  n_classes = cm.shape[0]

  # Calculate p_o (the observed proportionate agreement) and
  # p_e (the probability of random agreement)
  identity = np.identity(n_classes)
  p_o = np.sum((identity * cm) / n)
  p_e = np.sum((identity * expected) / n)

  # Calculate Cohen's kappa
  kappa = (p_o - p_e) / (1 - p_e)

  # Confidence intervals
  se = np.sqrt((p_o * (1 - p_o)) / (n * (1 - p_e)**2))
  ci = 1.96 * se * 2
  lower = kappa - 1.96 * se
  upper = kappa + 1.96 * se

  return (kappa, lower, upper)


In [8]:
graders = ["GPT", "Reviewer1", "Reviewer2", "Resolution"]
aiml_lists = [lst_reviewer_gpt_aiml, lst_reviewer1_aiml, lst_reviewer2_aiml, lst_resolution_aiml]
nlp_lists = [lst_reviewer_gpt_nlp, lst_reviewer1_nlp, lst_reviewer2_nlp, lst_resolution_nlp]

In [9]:
# AI/ML Kappas calculations for each unique pair of graders
for grader1_idx in range(0,len(graders)-1):
  for grader2_idx in range(grader1_idx+1, len(graders)):
    list1 = aiml_lists[grader1_idx]
    list2 = aiml_lists[grader2_idx]

    kappa, lower, upper = calculate_kappa_and_CI(list1, list2)

    print(f"Agreement between {graders[grader1_idx]} and {graders[grader2_idx]} on AI/ML parameter:")
    print(f"{kappa:.3f}, ({lower:.3f}, {upper:.3f})\n")


Agreement between GPT and Reviewer1 on AI/ML parameter:
0.901
0.901, (0.806, 0.996)

Agreement between GPT and Reviewer2 on AI/ML parameter:
0.899
0.899, (0.801, 0.996)

Agreement between GPT and Resolution on AI/ML parameter:
0.925
0.925, (0.841, 1.009)

Agreement between Reviewer1 and Reviewer2 on AI/ML parameter:
0.899
0.899, (0.801, 0.996)

Agreement between Reviewer1 and Resolution on AI/ML parameter:
0.975
0.975, (0.926, 1.024)

Agreement between Reviewer2 and Resolution on AI/ML parameter:
0.923
0.923, (0.837, 1.009)



In [10]:
# NLP Kappas calculations for each unique pair of graders
for grader1_idx in range(0,len(graders)-1):
  for grader2_idx in range(grader1_idx+1, len(graders)):
    list1 = nlp_lists[grader1_idx]
    list2 = nlp_lists[grader2_idx]

    kappa, lower, upper = calculate_kappa_and_CI(list1, list2)

    print(f"Agreement between {graders[grader1_idx]} and {graders[grader2_idx]} on NLP parameter:")
    print(f"{kappa:.3f}, ({lower:.3f}, {upper:.3f})\n")

Agreement between GPT and Reviewer1 on NLP parameter:
0.917
0.917, (0.824, 1.010)

Agreement between GPT and Reviewer2 on NLP parameter:
0.971
0.971, (0.915, 1.027)

Agreement between GPT and Resolution on NLP parameter:
0.944
0.944, (0.866, 1.021)

Agreement between Reviewer1 and Reviewer2 on NLP parameter:
0.890
0.890, (0.785, 0.996)

Agreement between Reviewer1 and Resolution on NLP parameter:
0.973
0.973, (0.920, 1.026)

Agreement between Reviewer2 and Resolution on NLP parameter:
0.917
0.917, (0.824, 1.010)

